
# Collaborative Filtering
### Body of code from Ibtesam Ahmed
#### Link: https://www.kaggle.com/code/ibtesama/getting-started-with-a-movie-recommendation-system/notebook


##### This model takes users movies and their ratings as input to train the model. When testing, the model is given the user and movie and the model will predict the rating (1-poor, 5-great).

#### No code start YET

In [ ]:
%pip install surprise

In [1]:
import pandas as pd
from surprise import Reader, Dataset, SVD, accuracy
from surprise.model_selection import train_test_split

#reader object to tell surprise how to interpret ratings (1-5)
reader = Reader()

# Loading ratings csv into ratings dataframe
ratings = pd.read_csv('tmdb/ratings_small.csv')

In [17]:
# View first five items
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [18]:
ratings.describe()

,userId,movieId,rating,timestamp
count,100004.000000,100004.000000,100004.000000,1.000040e+05
mean,347.011310,12548.664363,3.543608,1.129639e+09
std,195.163838,26369.198969,1.058064,1.916858e+08
min,1.000000,1.000000,0.500000,7.896520e+08
25%,182.000000,1028.000000,3.000000,9.658478e+08
50%,367.000000,2406.500000,4.000000,1.110422e+09
75%,520.000000,5418.000000,4.000000,1.296192e+09
max,671.000000,163949.000000,5.000000,1.476641e+09


## Splitting dataset

In [23]:
# Step 1: Split the raw DataFrame manually into train+val and test
trainval_df = ratings.sample(frac=0.8, random_state=42)  # 80% of data
test_df = ratings.drop(trainval_df.index)                # remaining 20%

# Step 2: Split train+val into train and validation
train_df = trainval_df.sample(frac=0.75, random_state=42)  # 75% of 80% = 60%
val_df = trainval_df.drop(train_df.index)                   # remaining 20%

# Step 3: Load each into Surprise Dataset
reader = Reader(rating_scale=(0.5, 5.0))
train_data = Dataset.load_from_df(train_df[['userId','movieId','rating']], reader)
val_data = Dataset.load_from_df(val_df[['userId','movieId','rating']], reader)
test_data = Dataset.load_from_df(test_df[['userId','movieId','rating']], reader)

# Step 4: Build trainsets / testsets for Surprise
trainset = train_data.build_full_trainset()
valset = val_data.build_full_trainset().build_testset()  # as a testset for evaluation
testset = test_data.build_full_trainset().build_testset()  # final testset

## Training model

In [24]:
# Define SVD (singular value decomposition) model.
svd = SVD()

# Train model
svd.fit(trainset)

## Validating model on val dataset

In [28]:
val_predictions = svd.test(valset)
accuracy.rmse(val_predictions)
accuracy.mae(val_predictions)

RMSE: 0.9050
MAE:  0.6944


0.6944381630233132

## Test model on test dataset

In [ ]:
# Test model on test set
test_predictions = svd.test(testset)
accuracy.rmse(test_predictions)
accuracy.mae(test_predictions)

## Testing single user

In [ ]:
# Inspect ratings for a specific user
ratings[ratings['userId'] == 1]

In [ ]:
# Predict user 1's prediction on movie 302
svd.predict(1, 302, 3)